##### Work on the cleaned sensor files 2018 and prepare the 5/10/15 min samples in HDF format. Also creates a adjcency list from the list of sensors and spatial embedding (SE) matrice and a timestamp afterwards.

In [12]:
import os
import pandas as pd
from geopy.distance import distance

# CONFIGURATION:

# Sensor file list, i.e., I15 NB and I15 SB
# Change with the list of sensor/sensors

sensor_merge_list = [
    "E:/I-15 Datataset/2018/cleaned/I-15 NB without Ramp _CLN.csv",
    "E:/I-15 Datataset/2018/cleaned/I-15 SB without Ramp _CLN.csv"
]

detectorlist_file_dir="../detector/detectors2018.csv"

sampled_file_save_dir= ""

sampling_rate=  ""      # 5T, 10T, 15T = 5 min , 10 mins , 15 mins 
####################################################################################

# Check if sensor list has more than one element.
# If more, then the sensor values will be concatenated, column-wise.

if len(sensor_merge_list) > 1:
    merged_df = []  # Initialize an empty list for storing DataFrames

    for file in sensor_merge_list:
        #file = os.path.normpath(file)
        temp_df = pd.read_csv(file)
        temp_df = temp_df.iloc[:, 1:] # Remove the index column
        merged_df.append(temp_df) # Store the DataFrame in the list

    # Concatenate the list of DataFrames column-wise
    his_data = pd.concat(merged_df, axis=1)

else:

    #file = os.path.normpath(sensor_merge_list[0])  # Get the first file in the list
    temp_df = pd.read_csv(file)
    his_data=temp_df

# Sampling 

if sampling_rate == "5T":
    his_data_resampled = his_data.iloc[::5, :]  # Select every 5th row
elif sampling_rate == "10T":
    his_data_resampled = his_data.iloc[::10, :]  # Select every 10th row
elif sampling_rate == "15T":
    his_data_resampled = his_data.iloc[::15, :]  # Select every 15th row
else:
    his_data_resampled = his_data  # If sampling_rate is empty or not recognized, keep the original data

# Saving his_data as csv    

# Task: 2 Create the graph from the sensor list solved using travelling salesman algo

# De-lane the detectors list csv

delane_df=pd.read_csv(detectorlist_file_dir, header=None)
delane_df = delane_df.drop_duplicates(subset=[5, 6], keep='first') # Group by lat lon and keep only the first occurrence of each group
delane_df.loc[:, 1] = delane_df.loc[:, 1].astype(str).str.rsplit('_', n=1).str[0] # Removing lane number from column 1
delane_df.iloc[:, 5] = delane_df.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
delane_df.iloc[:, 6] = delane_df.iloc[:, 6].astype(str).apply(lambda x: float(x[:4] + '.' + x[4:]))

# Using the delaned detectors values, filter the ones needed for lat lon positions from the main file
adj=pd.DataFrame()

for file in sensor_merge_list: 
    temp_df = pd.read_csv(file)
    temp_df=temp_df.iloc[:,1:] # remove first column (index column)
    mask_col=temp_df.columns

    # remove _d from each values of the temp_df
    mask_col=mask_col.astype(str).str.rsplit('d_', n=3).str[1]
    
    # Filter de laned sensor df using the temp_df_col to get the 
    temp_df = delane_df[delane_df.iloc[:, 1].isin(mask_col)]

    # Make a graph using tsp and then make an adj 
    temp_adj=get_adj_mat(temp_df)
    adj = pd.concat([adj, temp_adj])
    
# print(adj)


C:\Users\MSI\AppData\Local\Temp\ipykernel_14988\4238666586.py:30: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file)
C:\Users\MSI\AppData\Local\Temp\ipykernel_14988\4238666586.py:30: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file)
C:\Users\MSI\AppData\Local\Temp\ipykernel_14988\4238666586.py:70: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file)
C:\Users\MSI\AppData\Local\Temp\ipykernel_14988\4238666586.py:70: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file)


In [13]:
adj

,sensor1,sensor2,cost/m
0,401_1_435,401_2_437,367.391801
1,400_4_431,401_1_435,373.890103
2,400_3_430,400_4_431,495.974764
3,402_1_438,401_2_437,613.893428
4,400_1_423,400_3_430,1222.330759
...,...,...,...
48,433_2_354,433_1_353,893.424579
49,433_3_355,433_2_354,835.621116
50,434_1_356,433_3_355,1097.965890
51,434_2_357,434_1_356,1269.019442


In [17]:
# Map sensor names to interger based adjacency list

import pandas as pd

adj_tmp = adj.copy()

# Get all unique sensor names from both columns
all_sensors = pd.concat([adj_tmp['sensor1'], adj_tmp['sensor2']]).unique()

# Create a mapping between unique sensor names and unique integers starting from 1
sensor_to_integer = {sensor: i + 1 for i, sensor in enumerate(all_sensors)}

# Replace sensor names with integers in the DataFrame
adj_tmp['sensor1'] = adj_tmp['sensor1'].map(sensor_to_integer)
adj_tmp['sensor2'] = adj_tmp['sensor2'].map(sensor_to_integer)

# Save the DataFrame to a TSV file with tab separator
output_file_path = "E:/I-15 Datataset/2018/cleaned/prepared_datasets/I15NB-I15SB/adj_with_integers.txt"
adj_tmp.to_csv(output_file_path, index=False, sep='\t', header=False)

print("DataFrame saved with unique integers as Text")
adj_tmp


DataFrame saved with unique integers as Text


,sensor1,sensor2,cost/m
0,1,106,367.391801
1,2,1,373.890103
2,3,2,495.974764
3,4,106,613.893428
4,5,3,1222.330759
...,...,...,...
48,101,100,893.424579
49,102,101,835.621116
50,103,102,1097.965890
51,104,103,1269.019442


In [79]:
# Save the historical sensor data file as csv
# Save the historical sensor data file as hdf5

output_file_path = "E:/I-15 Datataset/2018/cleaned/prepared_datasets/I15NB-I15SB/ds.hdf"  # Replace with the desired output file path
# The 'key' parameter is used to give a name to the dataset inside the HDF5 file.
his_data.to_hdf(output_file_path, key='I15NB-SB', mode='w') 
print('Sensor data saved') 


c:\Users\MSI\AppData\Local\Programs\Python\Python311\Lib\site-packages\tables\path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'I15NB-SB'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


Sensor data saved


In [72]:
uncleaned_sensor_date_time=pd.read_csv("E:/I-15 Datataset/2018/dates.csv")

# Copying the uncleaned dataset to a new pd so that both copies remains
sensor_date_time=uncleaned_sensor_date_time.copy(deep=True)

# Remove first row
sensor_date_time=sensor_date_time.iloc[1:]

start_time_str = '18-01-18 09:55'
end_time_str = '18-01-18 11:21'

start_time = pd.to_datetime(start_time_str)
end_time = pd.to_datetime(end_time_str)
time_difference_minutes = (end_time - start_time).seconds/ 60

temp_date_time=[]
temp_index=[]

for minute in range(1,int((time_difference_minutes))):
   # Data-Time column data filling
   temp_date_time.append(start_time + pd.Timedelta(minutes=minute))
   # Index column data filling
   temp_index.append("18685" + "-" + str(minute))

# Formatting the list to DataFrames
temp_date_time=pd.DataFrame({"date_time":temp_date_time})
temp_index=pd.DataFrame({"date_index":temp_index})
# Concatenate two temp df side by side, axis=1
concatenated_df = pd.concat([temp_index, temp_date_time], axis=1)

concatenated_df=pd.concat([sensor_date_time[0:18684],concatenated_df,sensor_date_time[18684:373037]], axis=0)

In [ ]:
# get the timestamp column- second column
timestamps=concatenated_df.iloc[:, 1:2]
timestamps = pd.to_datetime(timestamps)
timestamps = timestamps.dt.strftime("%Y-%m-%d %H:%M:%S")
output_file_path = "E:/I-15 Datataset/2018/cleaned/prepared_datasets/I15NB-I15SB/timestamps.csv" 
timestamps.to_csv(output_file_path, index=False, header=True)
print("Timestamps file saved")

In [110]:
import pandas as pd

# Assuming you have the DataFrame 'concatenated_df' with the "Timestamp" column.

# Get the "Timestamp" column - second column (assuming it is the second column)
timestamps = concatenated_df.iloc[:, 1]

# Convert the "Timestamp" column to pandas datetime format
timestamps = pd.to_datetime(timestamps)

# Format the "Timestamp" column to the long date time format "%Y-%m-%d %H:%M:%S"
timestamps = timestamps.dt.strftime("%Y-%m-%d %H:%M:%S")

# Save the DataFrame to a CSV file with the formatted timestamps in long date time format
output_file_path = "E:/I-15 Datataset/2018/cleaned/prepared_datasets/I15NB-I15SB/timestamps.csv"
timestamps.to_csv(output_file_path, index=False, header=False)

print("Timestamps file saved")


Timestamps file saved


In [111]:
# Convert timestamps to datetime objects
import torch 

datetime_objects = pd.to_datetime(timestamps, format="%Y-%m-%d %H:%M:%S")

    # Extract day of the week (Monday=0, Sunday=6)
day_of_week = datetime_objects.dt.dayofweek

    # Extract time of the day in 15-minute intervals
time_of_day = (datetime_objects.dt.hour * 60 + datetime_objects.dt.minute) // 1

    # Assuming day_of_week and time_of_day are lists
time = torch.tensor(list(zip(day_of_week, time_of_day)))


In [113]:
import numpy as np
np.savetxt('tensor_data.txt', time.numpy(), delimiter=',')


In [9]:
# Create adj list using the list of sensors

import pandas as pd
import folium
from geopy.distance import distance

def calculate_distance(sensor_data):
    # Calculate distances between each pair of sensors
    num_sensors = len(sensor_data)
    distances = [[0.0] * num_sensors for _ in range(num_sensors)]
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            coord1 = (sensor_data.iloc[i][5], sensor_data.iloc[i][6])
            coord2 = (sensor_data.iloc[j][5], sensor_data.iloc[j][6])
            dist = distance(coord1, coord2).meters
            distances[i][j] = dist
            distances[j][i] = dist
    return distances

def get_adj_mat(sensor_data):
    distances = calculate_distance(sensor_data)
    
    # Create an empty list to store the adjacency list
    adjacency_list = []

    # Connect sensors with the shortest distance
    num_sensors = len(sensor_data)
    connected = set()
    min_distance = float('inf')
    min_i, min_j = -1, -1

    # Find the pair of sensors with the shortest distance
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            if distances[i][j] < min_distance:
                min_distance = distances[i][j]
                min_i, min_j = i, j

    # Connect the pair of sensors
    if min_i != -1 and min_j != -1:
        adjacency_list.append([sensor_data.iloc[min_i][1], sensor_data.iloc[min_j][1], min_distance])
        connected.add(min_i)
        connected.add(min_j)

    # Connect the remaining sensors
    while len(connected) < num_sensors:
        min_distance = float('inf')
        min_i = -1
        for i in range(num_sensors):
            if i not in connected:
                for j in connected:
                    if distances[i][j] < min_distance:
                        min_distance = distances[i][j]
                        min_i = i
                        min_j = j
        if min_i != -1:
            adjacency_list.append([sensor_data.iloc[min_i][1], sensor_data.iloc[min_j][1], min_distance])
            connected.add(min_i)

    # Create a DataFrame from the adjacency list
    adjacency_df = pd.DataFrame(adjacency_list, columns=['sensor1', 'sensor2', 'cost/m'])
    return adjacency_df

In [6]:
# Plot the adj graph on the map

import pandas as pd
import folium
from geopy.distance import distance

def calculate_distance(sensor_data):
    # Calculate distances between each pair of sensors
    num_sensors = len(sensor_data)
    distances = [[0.0] * num_sensors for _ in range(num_sensors)]
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            coord1 = (sensor_data.iloc[i][5], sensor_data.iloc[i][6])
            coord2 = (sensor_data.iloc[j][5], sensor_data.iloc[j][6])
            dist = distance(coord1, coord2).meters
            distances[i][j] = dist
            distances[j][i] = dist
    return distances

def get_graph(sensor_data):
    
    distances = calculate_distance(sensor_data)
    map = folium.Map(location=[sensor_data[5].mean(), sensor_data[6].mean()], zoom_start=10)

    # Connect sensors with the shortest distance
    num_sensors = len(sensor_data)
    connected = set()
    min_distance = float('inf')
    min_i, min_j = -1, -1

    # Find the pair of sensors with the shortest distance
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            if distances[i][j] < min_distance:
                min_distance = distances[i][j]
                min_i, min_j = i, j

    # Connect the pair of sensors with a polyline
    if min_i != -1 and min_j != -1:
        sensor1 = sensor_data.iloc[min_i]
        sensor2 = sensor_data.iloc[min_j]
        folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                        color='red').add_to(map)
        connected.add(min_i)
        connected.add(min_j)

    # Connect the remaining sensors
    while len(connected) < num_sensors:
        min_distance = float('inf')
        min_i = -1
        for i in range(num_sensors):
            if i not in connected:
                for j in connected:
                    if distances[i][j] < min_distance:
                        min_distance = distances[i][j]
                        min_i = i
                        min_j = j
        if min_i != -1:
            sensor1 = sensor_data.iloc[min_i]
            sensor2 = sensor_data.iloc[min_j]
            folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                            color='red').add_to(map)
            connected.add(min_i)

    
    for _, sensor in sensor_data.iterrows():
        tooltip_text = f"Sensor_ID: {sensor[1]}, Des: {sensor[3]}"
        folium.CircleMarker(
            location=[sensor[5], sensor[6]],
            radius=5,
            color='blue',
            fill=True,
            fill_color='blue',
            tooltip=tooltip_text  # Add tooltip
            ).add_to(map)
    return map